In [1]:
# Note: Cell is tagged to not show up in the mkdocs build
%load_ext autoreload
%autoreload 2

<div class="admonition abstract highlight">
    <p class="admonition-title">In short</p>
    <p>This tutorial shows how to create datasets with more advanced data-modalities through the .zarr format.</p>
</div>

## Pointer columns

Not all data might fit the tabular format, e.g. images or conformers. In that case, we have _pointer_ columns. Pointer columns do not contain the data itself, but rather store a reference to an external file from which the content can be loaded.

For now, we only support `.zarr` files as references. To learn more about `.zarr`, visit their documentation. Their [tutorial](https://zarr.readthedocs.io/en/stable/tutorial.html) specifically is a good read to better understand the main features. 

### Dummy example
For the sake of simplicity, let's assume we have just two datapoints. We will use this to demonstrate the idea behind pointer columns. 

In [2]:
import zarr
import platformdirs

import numpy as np
import datamol as dm
import pandas as pd

SAVE_DIR = dm.fs.join(platformdirs.user_cache_dir(appname="polaris-tutorials"), "002")

/mnt/ps/home/CORP/lu.zhu/miniconda3/envs/po_datasets/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Create two images and save them to a Zarr archive
base_path = dm.fs.join(SAVE_DIR, "data.zarr")
inp_col_name = "images"

images = np.random.random((2, 64, 64, 3))
root = zarr.open(base_path, "w")
root.array(inp_col_name, images)

<zarr.core.Array '/images' (2, 64, 64, 3) float64>

In [4]:
# consolidate
zarr.consolidate_metadata(base_path)

<zarr.hierarchy.Group '/'>

In [5]:
# For performance reasons, Polaris expects all data related to a column to be saved in a single Zarr array.
# To index a specific element in that array, the pointer path can have a suffix to specify the index.
train_path = f"{inp_col_name}#0"
test_path = f"{inp_col_name}#1"

In [6]:
tgt_col_name = "target"

table = pd.DataFrame(
    {
        inp_col_name: [train_path, test_path],  # Instead of the content, we specify paths
        tgt_col_name: np.random.random(2),
    }
)

In [7]:
from polaris.dataset import Dataset, ColumnAnnotation

dataset = Dataset(
    table=table,
    # To indicate that we are dealing with a pointer column here,
    # we need to annotate the column.
    annotations={"images": ColumnAnnotation(is_pointer=True)},
    # We also need to specify the path to the root of the Zarr archive
    zarr_root_path=base_path,
)

Note how the table does not contain the image data, but rather stores a path relative to the root of the Zarr. 

In [8]:
dataset.table.loc[0, "images"]

'images#0'

To load the data that is being pointed to, you can simply use the `Dataset.get_data()` utility method. 

In [9]:
dataset.get_data(col="images", row=0).shape

(64, 64, 3)

Creating a benchmark and the associated `Subset` objects will automatically do so! 

In [10]:
from polaris.benchmark import SingleTaskBenchmarkSpecification

benchmark = SingleTaskBenchmarkSpecification(
    dataset=dataset,
    input_cols=inp_col_name,
    target_cols=tgt_col_name,
    metrics="mean_absolute_error",
    split=([0], [1]),
)

In [11]:
train, test = benchmark.get_train_test_split()

for x, y in train:
    # At this point, the content is loaded from the path specified in the table
    print(x.shape)

(64, 64, 3)


## Creating datasets from `.zarr` arrays

While the above example works, creating the table with all paths from scratch is time-consuming when datasets get large. Instead, you can also automatically parse a Zarr archive into the expected tabular data structure. 

A Zarr archive can contain groups and arrays, where each group can again contain groups and arrays. Within Polaris, we expect the root to be a flat hierarchy that contains a single array per column.


### A single array for _all_ datapoints 

Polaris expects a flat zarr hierarchy, with a single array per pointer column: 
```
/
  column_a
```

Which will get parsed into a table like: 

| column_a          |
| ----------------- |
| column_a/array#1  |
| column_a/array#2  |
|       ...         |
| column_a/array#N  |

<div class="admonition info highlight">
    <p class="admonition-title">Note</p>
    <p>Notice the # suffix in the path, which indicates the index at which the data-point is stored within the big array. </p>
</div>

In [12]:
# Let's first create some dummy dataset with 1000 64x64 "images"
images = np.random.random((1000, 64, 64, 3))

In [13]:
path = dm.fs.join(SAVE_DIR, "zarr", "data.zarr")

with zarr.open(path, "w") as root:
    root.array(inp_col_name, images)

To create a dataset from a Zarr archive, we can use the convenience function `create_dataset_from_file()`.

In [14]:
from polaris.dataset import create_dataset_from_file

# Because Polaris might restructure the Zarr archive,
# we need to specify a location to save the Zarr file to.
dataset = create_dataset_from_file(path, zarr_root_path=dm.fs.join(SAVE_DIR, "zarr", "processed.zarr"))

# The path refers to the original zarr directory we created in the above code block
dataset.table.iloc[0][inp_col_name]

'images#0'

In [15]:
dataset.get_data(col=inp_col_name, row=0).shape

(64, 64, 3)

## Saving the dataset

We can still easily save the dataset. All the pointer columns will be automatically updated. 

In [16]:
savedir = dm.fs.join(SAVE_DIR, "json")
json_path = dataset.to_json(savedir)

2024-07-21 01:45:05.451 | INFO     | polaris._mixins:md5sum:27 - Computing the checksum. This can be slow for large datasets.
Finding all files in the Zarr archive:   0%|          | 0/131 [00:00<?, ?it/s]

Finding all files in the Zarr archive: 100%|██████████| 131/131 [00:00<00:00, 270.89it/s]
2024-07-21 01:45:05.952 | INFO     | polaris.dataset._dataset:to_json:426 - Copying Zarr archive to /mnt/ps/home/CORP/lu.zhu/.cache/polaris-tutorials/002/json/data.zarr. This may take a while.


In [ ]:
fs = dm.fs.get_mapper(path).fs
fs.ls(SAVE_DIR)

Besides the `table.parquet` and `dataset.yaml`, we can now also see a `data` folder which stores the content for the additional content from the pointer columns.

## Load the dataset

In [18]:
Dataset.from_json(json_path)

2024-07-21 01:45:21.629 | INFO     | polaris._mixins:md5sum:27 - Computing the checksum. This can be slow for large datasets.
Finding all files in the Zarr archive:   0%|          | 0/131 [00:00<?, ?it/s]

Finding all files in the Zarr archive: 100%|██████████| 131/131 [00:00<00:00, 252.96it/s]


{
  "name": null,
  "description": "",
  "tags": [],
  "user_attributes": {},
  "owner": null,
  "polaris_version": "dev",
  "default_adapters": {},
  "zarr_root_path": "/mnt/ps/home/CORP/lu.zhu/.cache/polaris-tutorials/002/zarr/processed.zarr",
  "readme": "",
  "annotations": {
    "images": {
      "is_pointer": true,
      "modality": "UNKNOWN",
      "description": null,
      "user_attributes": {},
      "dtype": "object"
    }
  },
  "source": null,
  "license": null,
  "curation_reference": null,
  "cache_dir": "/mnt/ps/home/CORP/lu.zhu/.cache/polaris/datasets/359ed002-48b1-4de7-a3df-83496a00bd6d",
  "md5sum": "ca4766d15dce67159a7f15b239215ca3",
  "artifact_id": null,
  "zarr_md5sum_manifest": [
    {
      "path": ".zgroup",
      "md5sum": "e20297935e73dd0154104d4ea53040ab",
      "size": 24
    },
    {
      "path": ".zmetadata",
      "md5sum": "a044a50b6c0e5548b0dc7df18947379d",
      "size": 736
    },
    {
      "path": "images/.zarray",
      "md5sum": "dc17c5e9998b27bb705a75ad33cb909a",
      "size": 386
    },
    {
      "path": "images/0.0.0.0",
      "md5sum": "c7d90c5bcfe8db29f7117f26eacc02d5",
      "size": 896585
    },
    {
      "path": "images/0.0.0.1",
      "md5sum": "a99677c8849c9b372b1b5ac225676c88",
      "size": 887100
    },
    {
      "path": "images/0.0.1.0",
      "md5sum": "b2fdcf48167d167071f2275a0932f1b5",
      "size": 896573
    },
    {
      "path": "images/0.0.1.1",
      "md5sum": "f9465617ff41934c559ff229ae7a25c1",
      "size": 886885
    },
    {
      "path": "images/0.0.2.0",
      "md5sum": "208bded7d07b3bbf3d7403fdc68594c1",
      "size": 896591
    },
    {
      "path": "images/0.0.2.1",
      "md5sum": "740f3cb027d289fb13b9371df510de9d",
      "size": 886831
    },
    {
      "path": "images/0.0.3.0",
      "md5sum": "3a922739a6a404f4124b4a8698eb3775",
      "size": 896576
    },
    {
      "path": "images/0.0.3.1",
      "md5sum": "97c32de6051a4b34d5835f07c46c1b48",
      "size": 887198
    },
    {
      "path": "images/0.1.0.0",
      "md5sum": "d16a08610a67f79d5536e56a4e1cce85",
      "size": 896576
    },
    {
      "path": "images/0.1.0.1",
      "md5sum": "95c50d28dad41849339c72749fff47e6",
      "size": 886739
    },
    {
      "path": "images/0.1.1.0",
      "md5sum": "47a9d07c956a513d52de739f918633c5",
      "size": 896591
    },
    {
      "path": "images/0.1.1.1",
      "md5sum": "425c363aefc7a2ea30904ff71c23533b",
      "size": 886831
    },
    {
      "path": "images/0.1.2.0",
      "md5sum": "b4f690ef587de25767003647e0df1043",
      "size": 896590
    },
    {
      "path": "images/0.1.2.1",
      "md5sum": "0eae1e2d389c7758d50b1d14b6a892f6",
      "size": 887072
    },
    {
      "path": "images/0.1.3.0",
      "md5sum": "88ee357e3e19f8c2ab6e26a084405b95",
      "size": 896591
    },
    {
      "path": "images/0.1.3.1",
      "md5sum": "7df80f5c40676dbf5763b75360800cdb",
      "size": 887221
    },
    {
      "path": "images/0.2.0.0",
      "md5sum": "2535c0398749e784ca4cd8ec8e9e6852",
      "size": 896564
    },
    {
      "path": "images/0.2.0.1",
      "md5sum": "180675637d84bb237c87b1caae384a4b",
      "size": 887093
    },
    {
      "path": "images/0.2.1.0",
      "md5sum": "99301544fe15b3c503de549edb78597a",
      "size": 896586
    },
    {
      "path": "images/0.2.1.1",
      "md5sum": "5b007a4b863be52d2d507aca35a39e62",
      "size": 887114
    },
    {
      "path": "images/0.2.2.0",
      "md5sum": "9563c95e266dfa70a33f43692a2ec77b",
      "size": 896583
    },
    {
      "path": "images/0.2.2.1",
      "md5sum": "ebd9aef6bde85ac1a554332e53512a71",
      "size": 887189
    },
    {
      "path": "images/0.2.3.0",
      "md5sum": "6856d28761a759a3c257073315f4c635",
      "size": 896586
    },
    {
      "path": "images/0.2.3.1",
      "md5sum": "26b26d3cd91dacc4858d99340c54fa20",
      "size": 887048
    },
    {
      "path": "images/0.3.0.0",
      "md5sum": "0843993eba6130bf4e0b5c90f783c53d",
      "size": 896570
    },
    

### Upload zarr dataset to Hub

In [23]:
# Define the zarr dataset metadata before uploading
dataset.name = "tutorial_zarr"
dataset.license = "CC-BY-4.0"
dataset.source = "https://github.com/polaris-hub/polaris"

In [24]:
dataset.upload_to_hub(owner="polaris")

⠸ Uploading dataset... 

2024-07-21 01:49:16.093 | INFO     | polaris.hub.client:upload_dataset:602 - Copying Zarr archive to the Hub. This may take a while.


✅ SUCCESS: Your dataset has been successfully uploaded to the Hub. View it here: https://polarishub.io/datasets/polaris/tutorial_zarr
 


/mnt/ps/home/CORP/lu.zhu/miniconda3/envs/po_datasets/lib/python3.12/site-packages/yaspin/core.py:228: UserWarning: color, on_color and attrs are not supported when running in jupyter
  self._color = self._set_color(value) if value else value


The End. 